# Object → Colors

Extract Object Names & Color distribution from caption + image.

### "Visualize Each Stage of Pipeline!"

####  Incremental Development

- Objects with `uni`, `bi`, `tri` & `any`:  **1000**
- Objects with Qualifiers (context, state): **3000**
- Refine names via embedding clusters:    **5000**
- Later use Entropy for 'object selection'
  (e.g. shoes: party shoes vs tennis shoes)

Leverage *ConceptNet-KB* for verifying object names.

----


#### Object Name Extraction

- Fetch object names (and frequency) using POS Tags
- Track image & region IDs for each object name
- Maintain a flag indicating the presence of object name in ConceptNet

In [1]:
import requests
import numpy as np
from tqdm import tqdm
from os.path import join as osj
from utils import read_json, save_json
from data_dev import ColorDataDev
from collections import OrderedDict, Counter

DATA_DIR = '../../../Datasets/Visual_Genome'

In [2]:
raw_data = read_json(osj(DATA_DIR, 'region_graphs.json'))

In [3]:
caps_img = [len(x['regions']) for x in raw_data]

_ = int(np.mean(caps_img))
__ = int(np.std(caps_img))

f"Captions per Image: µ = {_}, σ = {__}"



'Captions per Image: µ = 50, σ = 9'

## Region Data

In [4]:
def _print(l: list):
    for e in l:
        print(e)
    print()

_idx = 0

d = raw_data[_idx]['regions']

for x in d:
    print(x['phrase'], '\n')

    print('Objects:')
    _print(x['objects'])

    print('Synsets:')
    _print(x['synsets'])

    print('Relations:')
    _print(x['relationships'])
    print('\n--------------------\n')

the clock is green in colour 

Objects:
{'name': 'clock', 'h': 339, 'object_id': 1058498, 'synsets': ['clock.n.01'], 'w': 79, 'y': 91, 'x': 421}

Synsets:
{'entity_idx_start': 4, 'entity_idx_end': 9, 'entity_name': 'clock', 'synset_name': 'clock.n.01'}
{'entity_idx_start': 22, 'entity_idx_end': 28, 'entity_name': 'colour', 'synset_name': 'color.n.04'}

Relations:


--------------------

shade is along the street  

Objects:
{'name': 'street', 'h': 262, 'object_id': 5046, 'synsets': ['street.n.01'], 'w': 714, 'y': 328, 'x': 77}
{'name': 'shade', 'h': 192, 'object_id': 5045, 'synsets': ['shade.n.01'], 'w': 274, 'y': 338, 'x': 119}

Synsets:
{'entity_idx_start': 0, 'entity_idx_end': 5, 'entity_name': 'shade', 'synset_name': 'shade.n.01'}
{'entity_idx_start': 19, 'entity_idx_end': 25, 'entity_name': 'street', 'synset_name': 'street.n.01'}

Relations:
{'synsets': ['along.r.01'], 'predicate': 'ON', 'relationship_id': 15927, 'object_id': 5046, 'subject_id': 5045}


--------------------

man i

#### Map Names (synset → entities)


In [5]:
synset2names = {}

for d in raw_data:
    for x in d['regions']:
        for s in x['synsets']:
            # Entity Name
            name = s['entity_name']
            # Lowercase
            name = name.lower()

            if s['synset_name'] not in synset2names:
                synset2names[s['synset_name']] = set()

            synset2names[s['synset_name']].add(name)

sum([len(v) for k,v in synset2names.items()]), len(synset2names)

(19440, 11930)

In [8]:
synset2names = OrderedDict(sorted(synset2names.items()))

# for syn, names in synset2names.items():
#     print(syn, names)

synset2names = {syn: list(names) for syn, names in synset2names.items()}

# save_json(synset2names, path='./temp/syn2ent.json')

In [10]:
synset_freq = []

for d in raw_data:
    for x in d['regions']:
        for s in x['synsets']:
            synset_freq.append(s['synset_name'])

synset_freq = Counter(synset_freq)
synset_freq = sorted(synset_freq.items(), key=lambda x: x[1], reverse=True)
synset_freq = OrderedDict(synset_freq)

# save_json(synset_freq, path='./temp/syn_freq.json')

In [ ]:
_f = [f for s, f in synset_freq]
print('Freq: {:.2f} ({:.2f})'.format(np.mean(_f), np.std(_f)))


<br>
<br>

## Attributes Data

In [2]:
raw_data = read_json(osj(DATA_DIR, 'attributes.json'))

In [7]:
objs_img = [len(d['attributes']) for d in tqdm(raw_data)]

_ = int(np.mean(objs_img))
__ = int(np.std(objs_img))

f"Objects per Image: µ = {_}, σ = {__}"

100%|██████████| 108077/108077 [00:00<00:00, 2019980.19it/s]


'Objects per Image: µ = 35, σ = 20'

In [8]:
attrs_img = []

for d in tqdm(raw_data):
    num_attrs = 0
    for x in d['attributes']:
        if 'attributes' in x:
            num_attrs += 1

    attrs_img.append(num_attrs)

_ = int(np.mean(attrs_img))
__ = int(np.std(attrs_img))

f"Attributes per Image: µ = {_}, σ = {__}"

100%|██████████| 108077/108077 [00:00<00:00, 228233.39it/s]


'Attributes per Image: µ = 15, σ = 9'

In [4]:
# Explore an Image

_idx = 2

d = raw_data[_idx]['attributes']

for x in d:
    name = x['names'][0]
    if 'attributes' in x:
        print(name, ': ', x['attributes'])
    else:
        print(name)

office phone :  ['multi-line phone']
outlet :  ['electrical']
outlet :  ['data line']
keyboard :  ['white']
monitor :  ['white', 'switched off', 'turned off']
cpu :  ['computer tower']
desktop :  ['curved']
filing cabinet
drawer
drawer
computer case :  ['black', 'leather']
mouse :  ['white']
wireless phone
office phone base
desk
multiple outlet
plug
plug
plug
plug
girl :  ['sitting']
monitor :  ['off']
wall
hair :  ['long']
chain
pen :  ['blue', 'yellow', 'black']
cable
floor
cable
pluged
bag :  ['black']
wall
black hair
telephone :  ['black']
white top
strap :  ['black']
cubicles
dividing screen
picture
picture :  ['hanged']
computer
cables
wall
photos
table :  ['white']
handle
floor
necklace
grey keys
desk
computer tower
cabinet :  ['grey', 'beige']


In [9]:
_names = []

multi_name_data = []

for d in raw_data:
    for x in d['attributes']:
        n = x['names']
        if len(n) > 1:
            multi_name_data.append(x)
        _names.append(len(n))

_ = np.mean(_names)
__ = np.std(_names)

f"Names per sample: µ = {_:.2f}, σ = {__:.2f}"

'Names per sample: µ = 1.02, σ = 0.19'

#### Map Attributes (one:many) → Object: Attributes

In [9]:
obj2attr = {}

for d in raw_data:
    for x in d['attributes']:
        if 'attributes' in x:
            for name in x['names']:
                if name not in obj2attr:
                    obj2attr[name] = set()
                for attr in x['attributes']:
                    attr = attr.lower()
                    obj2attr[name].add(attr)

_attr_per_obj = [len(a) for o, a in obj2attr.items()]

# Sort by num attributes
obj2attr = OrderedDict(sorted(obj2attr.items(), key=lambda x: len(x[1])))

_ = np.mean(_attr_per_obj)
__ = np.std(_attr_per_obj)

f"Attributes per Object: µ = {_:.2f}, σ = {__:.2f}"

'Attributes per Object: µ = 8.08, σ = 53.75'

#### Filter Objects with ConceptNet

In [1]:
# save_json(obj2attr_cn, path='./temp/object_with_attr_conceptnet.json')

'TODO'

In [ ]:
# Sort objects based on entropy (skewness) of color (i.e. school bus > jeans)
